In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
energy_data = pd.read_excel('ProblemCData.xlsx')

In [5]:
energy_data.head(100)

,MSN,StateCode,Year,Data
0,ABICB,AZ,1960,0.00000
1,ABICB,AZ,1961,0.00000
2,ABICB,AZ,1962,0.00000
3,ABICB,AZ,1963,0.00000
4,ABICB,AZ,1964,0.00000
5,ABICB,AZ,1965,0.00000
6,ABICB,AZ,1966,0.00000
7,ABICB,AZ,1967,0.00000
8,ABICB,AZ,1968,0.00000
9,ABICB,AZ,1969,0.00000
